In [ ]:
import json
import os
import pptxSections
import pptxTools
import jsonProcessing
import serverS3
import boto3

In [ ]:
def cv_creation(consultant):
    serverS3.download_files_s3()
    prs = pptxTools.init_pptx(7.5, 13.33)
    slide = pptxTools.init_slide(prs)
    pptxSections.name_section(slide, consultant)
    if consultant.description != "":
        section_height_left = section_height_right = pptxSections.description_section(slide, consultant)
        section_height_left = pptxSections.skills_section(slide, consultant, section_height_left)
    else:
        section_height_left = pptxSections.skills_section(slide, consultant, 1)
        section_height_right = 1
    if consultant.education != []:
        section_height_left = pptxSections.education_section(slide, consultant, section_height_left)
    if consultant.sectors != []:
        pptxSections.sectors_section(slide, consultant, section_height_left)
    pptxSections.jobs_section(slide, consultant, section_height_right)
    ppt_name = "[mini CV - FR] " + consultant.name + ".pptx"
    prs.save(ppt_name)
    serverS3.upload_files_s3(ppt_name)

In [ ]:
def lambda_handler(event, context):
    os.chdir('/tmp')
    event_body = json.loads(event["body"])
    consultant = jsonProcessing.obj_consultant(event_body)
    cv_creation(consultant)
    return {
        "statusCode": 200,
        "headers": {
            "Content-Type": "application/json ; charset=UTF-8"
        },
        "body": json.dumps(serverS3.create_presigned_post(consultant.name))
    }
